In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.astrosage.com/celebrity-horoscope/'
response = requests.get(url)
response

<Response [200]>

In [18]:
if response.status_code==200:
    print('ok')

ok


In [4]:
soup = BeautifulSoup(response.text,'html.parser')
browse = soup.find('div',{'class': "ui-alphabetic"})


In [22]:
alphas = browse.find_all('li')
alphas


[<li><a href="default.asp?ch=A" title="A"><span>A</span></a></li>,
 <li><a href="default.asp?ch=B" title="B"><span>B</span></a></li>,
 <li><a href="default.asp?ch=C" title="C"><span>C</span></a></li>,
 <li><a href="default.asp?ch=D" title="D"><span>D</span></a></li>,
 <li><a href="default.asp?ch=E" title="E"><span>E</span></a></li>,
 <li><a href="default.asp?ch=F" title="F"><span>F</span></a></li>,
 <li><a href="default.asp?ch=G" title="G"><span>G</span></a></li>,
 <li><a href="default.asp?ch=H" title="H"><span>H</span></a></li>,
 <li><a href="default.asp?ch=I" title="I"><span>I</span></a></li>,
 <li><a href="default.asp?ch=J" title="J"><span>J</span></a></li>,
 <li><a href="default.asp?ch=K" title="K"><span>K</span></a></li>,
 <li><a href="default.asp?ch=L" title="L"><span>L</span></a></li>,
 <li><a class="last" href="default.asp?ch=M" title="M"><span>M</span></a></li>,
 <li><a href="default.asp?ch=N" title="N"><span>N</span></a></li>,
 <li><a href="default.asp?ch=O" title="O"><span>O

In [24]:
pages=[36,18,17,20,9,6,13,10,4,36,18,10,32,12,2,20,1,30,40,13,2,10,5,1,2,2] # pages for A,B, C,...N.....Z
for i,j in enumerate(pages):
    print(i,j)

0 36
1 18
2 17
3 20
4 9
5 6
6 13
7 10
8 4
9 36
10 18
11 10
12 32
13 12
14 2
15 20
16 1
17 30
18 40
19 13
20 2
21 10
22 5
23 1
24 2
25 2


In [25]:
pages=[36,18,17,20,9,6,13,10,4,36,18,10,32,12,2,20,1,30,40,13,2,10,5,1,2,2]# pages for A,B, C,...N.....Z
all_url=[]
for i,page in enumerate(pages):
    person_list=[]
    for p in range(1,page+1):
        new_url = url + alphas[i].a['href'] + '&page='+str(p)
        new_response = requests.get(new_url)

        new_soup = BeautifulSoup(new_response.text,'html.parser')
        table=new_soup.find('table')
        persons=table.find_all('a', title=True, style=False)

        person_list.extend(persons)

    for i in range(len(person_list)):
        url3 = url + person_list[i]['href']
        print(url3)
        all_url.append(url3)
        

df= pd.DataFrame(all_url)
df.to_csv('all_data_links.csv',index=False,header=['Links'])
                

KeyboardInterrupt: 

In [28]:
df = pd.read_csv('all_data_links.csv')
df.head()

,Links
0,https://www.astrosage.com/celebrity-horoscope/...
1,https://www.astrosage.com/celebrity-horoscope/...
2,https://www.astrosage.com/celebrity-horoscope/...
3,https://www.astrosage.com/celebrity-horoscope/...
4,https://www.astrosage.com/celebrity-horoscope/...


In [34]:
all_url = list(df['Links'])

In [ ]:

list_of_dict=[]
f=1
for i in all_url:
    try:
        response = requests.get(i)
        soup3 = BeautifulSoup(response.text,'html.parser')
        info = soup3.find('div',{'class':'ui-cbcol-l'})
        celeb=info.find_all('div', class_ = 'celebcont')
        d ={}
        for i in range(len(celeb)):
            text = celeb[i].get_text()
            features=text.strip('\n').split('\n')
            features[0]=''.join(e for e in features[0] if e.isalnum())
            if features[0] in ('Longitude', 'Latitude'):
                features[1] = features[1].replace(u'\xa0', u' ')
            d[features[0]]=features[1]
        cloud = soup3.find('form',{'name':'AstroCloudFrm'})
        di ={}
        for c in cloud.find_all('input') :
            di[c.get('name')]=c.get('value')
        dic={**d,**di}
        list_of_dict.append(dic)
    except:
        pass
    print(f)
    f=f+1


In [ ]:
new_list=[]
for i in list_of_dict:
    new_list.append({k.lower():v for k,v in i.items()})

In [ ]:
import csv
with open('astro_data.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = [i.lower() for i in list(dic.keys())] )
    writer.writeheader()
    writer.writerows(new_list)

In [2]:
pd.read_csv('astro_data.csv')

,name,dateofbirth,timeofbirth,placeofbirth,longitude,latitude,timezone,informationsource,astrosagerating,cloudbtn,...,longdeg,latmin,longmin,longew,latns,dst,languagecode,ayanamsa,kphn,charting
0,Amisha Patel,"Monday, June 9, 1975",00:00:00,Mumbai,72 E 52,19 N 4,5.50,Dirty Data,Dirty Data,See kundli in AstroSage Cloud,...,72,4,52,E,N,0,0,0,0,0
1,Akshar Patel,"Thursday, January 20, 1994",12:00:00,Anand,72 E 55,22 N 34,5.50,Dirty Data,Dirty Data,See kundli in AstroSage Cloud,...,72,34,55,E,N,0,0,0,0,0
2,Anand Ahuja,"Thursday, July 7, 1983",00:00:00,Delhi,77 E 13,28 N 40,5.50,Dirty Data,Dirty Data,See kundli in AstroSage Cloud,...,77,40,13,E,N,0,0,0,0,0
3,Ameet Chana,"Friday, September 12, 1975",12:00:00,Surrey,122 W 49,49 N 6,-8.00,Dirty Data,Dirty Data,See kundli in AstroSage Cloud,...,122,6,49,W,N,0,0,0,0,0
4,Alia Bhatt,"Monday, March 15, 1993",04:10:00,London,0 W 7,51 N 30,0.00,Reference,Reference,See kundli in AstroSage Cloud,...,0,30,7,W,N,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7013,Zakir Hussain,"Monday, February 08, 1897",15:31:15,Hyderabad,78 E 26,17 N 22,5.50,Kundli Sangraha (Bhat),Accurate,See kundli in AstroSage Cloud,...,78,22,26,E,N,0,0,0,0,0
7014,Zooey Deschanel,"Thursday, January 17, 1980",12:00:00,Los Angeles,118 W 15,34 N 0,-5.00,Unknown,Dirty Data,See kundli in AstroSage Cloud,...,118,0,15,W,N,0,0,0,0,0
7015,Zachary Taylor,"Wednesday, November 24, 1784",10:56:51,"79 W 7, 37 N 50",79 W 7,37 N 50,-5.25,Unknown,Dirty Data,See kundli in AstroSage Cloud,...,79,50,7,W,N,0,0,0,0,0
7016,Zucchero,"Sunday, September 25, 1955",18:14:59,"10 E 35, 44 N 42",10 E 35,44 N 42,1.00,Unknown,Dirty Data,See kundli in AstroSage Cloud,...,10,42,35,E,N,0,0,0,0,0
